[2024-07-26 Fiddler](https://thefiddler.substack.com/p/can-you-even-the-odds)
====================
Let $w$ be your probability of winning when it's your turn to roll.  Consider the AB|AB|AB order of play.

Let $p = 1/6$ be the probability of winning a roll.

In [1]:
%display latex
w, p = var('w p')
eqn1 = w == p + (1-p)*(1-w)
eqn1

w == (p - 1)*(w - 1) + p

In [2]:
solve(eqn1, w)

[w == -1/(p - 2)]

In [3]:
numerical_approx(solve(eqn1,w)[0].rhs()(p=1/6))

0.545454545454545

So player A will win approximately 54.55% of the time.

Now consider the AB|BA|AB order of play.  Consider what happens if the first roll does not win.

In [4]:
eqn2 = w == p + (1-p)*(p + (1-p)*(1-w))
eqn2

w == -((p - 1)*(w - 1) + p)*(p - 1) + p

In [5]:
solve(eqn2, w)

[w == (1/(p^2 - 2*p + 2))]

Now, add in the first roll.

In [6]:
w_after_first_roll = solve(eqn2, w)[0].rhs()
w_abba = p + (1-p)*(1-w_after_first_roll)
w_abba

(p - 1)*(1/(p^2 - 2*p + 2) - 1) + p

In [7]:
numerical_approx(w_abba(p=1/6))

0.508196721311475

So player A will win approximately 50.82% of the time, which is fairer than AB|AB|AB.

[Simulations](20240726.go) agree:

    $ go run 20240726.go
    AB 5454278/10000000 0.545428
    ABBA 5083593/10000000 0.508359
    Thue-Morse 5016187/10000000 0.501619
    Round 1: A
    Round 2: B
    Round 3: BA
    Round 4: BAAB
    Round 5: BAABABBA
    Round 6: BAABABBAABBABAAB

Extra credit
------------

Let $W(n)$ be player A's probability of winning in round $n$, and $L(n)$ be player A's probability of losing in round $n$.

$$
\begin{aligned}
    W(1) &= p \\
    L(1) &= 0 \\
    W(2) &= 0 \\
    L(2) &= p \\
\end{aligned}
$$

For $n \ge 3$, the first half of the round is the previous round, and the second half the round is
the previous round reversed, giving the recurrence relation
$$
\begin{aligned}
    W(n+1) &= W(n) + (1-W(n)-L(n))L(n) \\
    L(n+1) &= L(n) + (1-W(n)-L(n))W(n) \\
\end{aligned}
$$

In [8]:
def W(n):
    if n == 1:
        return p
    elif n == 2:
        return 0
    else:
        return W(n-1) + (1-W(n-1)-L(n-1))*L(n-1)
def L(n):
    if n == 1:
        return 0
    elif n == 2:
        return p
    else:
        return L(n-1) + (1-W(n-1)-L(n-1))*W(n-1)

In [9]:
def sumW(maxN,n=1):
    if n >= maxN:
        return W(n)
    else:
        return W(n) + (1-W(n)-L(n))*sumW(maxN,n+1)

In [10]:
[sumW(i).factor() for i in range(1,6)]

[p,
 p,
 -(p^2 - p + 1)*(p - 2)*p,
 (p^3 - 3*p^2 + 2*p - 1)*(p^2 - 2*p + 2)*(p - 2)*p,
 -(p^8 - 7*p^7 + 21*p^6 - 36*p^5 + 39*p^4 - 28*p^3 + 14*p^2 - 4*p + 1)*(p^4 - 4*p^3 + 6*p^2 - 4*p + 2)*(p^2 - 2*p + 2)*(p - 2)*p]

In [11]:
[sumW(i)(p=1/6) for i in range(1,8)]

[1/6,
 1/6,
 341/1296,
 108031/279936,
 1339018029701/2821109907456,
 663397206470899569501151/1326443518324400147398656,
 31770605684172649673767341248855588372799862163141/63340286662973277706162286946811886609896461828096]

In [12]:
[numerical_approx(sumW(i)(p=1/6)) for i in range(8,12)]

[0.501590339167761, 0.501590339204269, 0.501590339204269, 0.501590339204269]

I think it's possible to solve the recurrence relation to get a closed-form expression that adds up the win probabilities over infinite rounds, but I don't know how to do that.  In any case, it certainly seems to converging to approximately 50.159%.